In [26]:
# imports

import cv2
from PIL import Image, ImageOps
import os
from sys import argv
import numpy as np
import chess
import chess.engine
from stockfish import Stockfish
import pytesseract
from PIL import ImageGrab
import pyautogui
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from stockfish import StockfishException
import time
%matplotlib inline
engine = chess.engine.SimpleEngine.popen_uci("stockfish_15.1_win_x64_avx2/stockfish-windows-2022-x86-64-avx2.exe")
stockfish = Stockfish('stockfish_15.1_win_x64_avx2/stockfish-windows-2022-x86-64-avx2.exe')
limit = chess.engine.Limit(time = 2.0)

In [27]:
pieces = {
    'black_pawn' : 'p',
    'black_pawn1' : 'p',
    'black_pawn2' : 'p',
    'black_pawn3' : 'p',
    'black_rook' : 'r',
    'black_rook1' : 'r',
    'black_rook2' : 'r',
    'black_rook3' : 'r',
    'black_knight' : 'n',
    'black_knight1' : 'n',
    'black_knight2' : 'n',
    'black_knight3' : 'n',
    'black_bishop' : 'b',
    'black_bishop1' : 'b',
    'black_bishop2' : 'b',
    'black_bishop3' : 'b',
    'black_queen' : 'q',
    'black_queen1' : 'q',
    'black_queen2' : 'q',
    'black_queen3' : 'q',
    'black_king' : 'k',
    'black_king1' : 'k',
    'black_king2' : 'k',
    'black_king3' : 'k',
    'black_box' : '1',
    'black_box1' : '1',
    
    'white_pawn' : 'P',
    'white_pawn1' : 'P',
    'white_pawn2' : 'P',
    'white_pawn3' : 'P',
    'white_rook' : 'R',
    'white_rook1' : 'R',
    'white_rook2' : 'R',
    'white_rook3' : 'R',
    'white_knight' : 'N',
    'white_knight1' : 'N',
    'white_knight2' : 'N',
    'white_knight3' : 'N',
    'white_bishop' : 'B',
    'white_bishop1' : 'B',
    'white_bishop2' : 'B',
    'white_bishop3' : 'B',
    'white_queen' : 'Q',
    'white_queen1' : 'Q',
    'white_queen2' : 'Q',
    'white_queen3' : 'Q',
    'white_king' : 'K',
    'white_king1' : 'K',
    'white_king2' : 'K',
    'white_king3' : 'K',
    'white_box' : '1',
    'white_box1' : '1'
}
layout = []

In [28]:
# gray-scale preprocessing

#using cv2
def preprocess(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#using Pillow
def preprocess1(image):
    return ImageOps.grayscale(image)

#using scikit
def preprocess2(image):
    return color.rgb2gray(image)

In [29]:
# root mean square evaluation of two images for comparison

def rmse(imageA, imageB):
    np.delete(imageA, np.s_[::2], 1)
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [30]:
# closest value approximation

def calc_closest_val(dict):
    result = {}
    closest = min(dict.values())
    for key, value in dict.items():
        if (value == closest):
            result[key] = closest
            
            #piece description
            key = key[8:-4]
            
            layout.append(pieces[key])

    return result

In [31]:
# image comparison

def compare(image):
    img = argv[1]
    main = cv2.imread(image)
    rmse_measures = {}
    scale_percent = 100 # percent of original img size
    width = int(main.shape[1] * scale_percent / 100)
    height = int(main.shape[0] * scale_percent / 100)
    dim = (width, height)

    data_dir = 'dataset'
   
    for file in os.listdir(data_dir):
        img_path = os.path.join(data_dir, file)
        data_img = cv2.imread(img_path)
        resized_img = cv2.resize(data_img, dim, interpolation = cv2.INTER_AREA)
        rmse_measures[img_path]= rmse(main, resized_img)

    rms = calc_closest_val(rmse_measures)

In [32]:
# breakig the chess board into 8x8 matrix

def splitter(path, rows, columns):
    img = cv2.imread(path)
    resized = cv2.resize(img, (400,400))
    width, height = resized.shape[1], resized.shape[0]
    stepx = int(width/columns)
    stepy = int(height/rows)
    for r in range(0, width, stepx):
        for c in range(0, height, stepy):
            cv2.imwrite(f"img{r}_{c}.jpg",resized[r:r+stepy, c:c+stepx, :])
            piece = f'img{r}_{c}.jpg'
            compare(piece)
            os.remove(piece)

In [33]:
# suggest next chess move

def next_move(fen_val):
    try:
        stockfish.set_fen_position(fen_val)
        print(stockfish.get_best_move())
    except StockfishException:
        pass
    else:
        board = chess.Board(fen_val)
        res = engine.play(board, limit)
        print(res.move)

In [34]:
# grab a particular part of screen

def board_grabber():
    left, top, width, height = 534, 130, 795, 793
    screenshot = pyautogui.screenshot(region=(left, top, width, height))
    screenshot.save('sample.jpg')

    image = mpimg.imread('sample.jpg')

In [35]:
# look for board on screen

def get_board():
    screenshot = pyautogui.screenshot()
    screenshot.save('dump.jpg')
    img = cv2.imread('dump.jpg', cv2.IMREAD_GRAYSCALE)
    assert img is not None, "File not found"
    img2 = img.copy()
    template = cv2.imread('board.jpg', cv2.IMREAD_GRAYSCALE)
    assert template is not None, "File not found"
    w, h = template.shape[::-1]

    methods = ['cv2.TM_CCOEFF_NORMED']
    for meth in methods:
        img = img2.copy()
        method = eval(meth)
        res = cv2.matchTemplate(img,template,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        if method in [cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv2.rectangle(img,top_left, bottom_right, 255, 2)
        img = img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        cv2.imwrite('sample.jpg', img)

In [36]:
# calculating fen value of a chess board from given array

def fen_calc():
    get_board()
    splitter('sample.jpg', 8, 8)
    board = [layout[i : i + 8] for i in range(0, len(layout), 8)]

    fen = ""
    for i, row in enumerate(board):
        empty_count = 0
        for j, square in enumerate(row):
            if square == "1":
                empty_count += 1
            else:
                if empty_count > 0:
                    fen += str(empty_count)
                    empty_count = 0
                fen += square
        if empty_count > 0:
            fen += str(empty_count)
        if i < 7:
            fen += "/"
    player = input("Who is to move? ")
    if player == 'b':
        fen = fen[::-1]
        fen += " b - - 0 1"
    elif player == 'w':
        fen += " w - - 0 1"
    else:
        print("Error")
    print(fen)
    next_move(fen)

In [37]:
# main function

#while True:
try:
    #time.sleep(2)
    fen_calc()
    #time.sleep(10)
except KeyboardInterrupt:
    print('exiting...')
    #break

Who is to move? b
2r4k/p2R2p1/1p4Qn/4p3/4P3/1q3BPp/7P/5R1K b - - 0 1
b3f3
b3f3
